<a href="https://colab.research.google.com/github/cmfritz/capstone/blob/main/data_clean.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
%cd /content/gdrive/My Drive/Github/capstone

/content/gdrive/My Drive/Github/capstone


In [ ]:
# import my custom library into colab
import sys
sys.path.append('/content/gdrive/MyDrive/Github/capstone/')
import mylibrary as mylib

In [ ]:
df = pd.read_pickle('data/master_data.pkl')
df_pats_w_sectors = pd.read_pickle('data/pats_w_sectors.pkl')

In [ ]:
df_master = mylib.merge_and_clean(df_pats_w_sectors, df)

In [ ]:
df_master.to_pickle('data/data_4_analysis.pkl')